<a href="https://colab.research.google.com/github/SusantaMondal22/Reusable-Assests-in-MachineLearning/blob/master/NLP/Sentiment_Classification_SeqNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Classification


## Loading the dataset

In [ ]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

Using TensorFlow backend.


17473536/17464789 [==============================] - 1s 0us/step


In [ ]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300 #number of word used from each review

## Train test split

In [ ]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

In [ ]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

('x_train shape:', (25000, 300))
('x_test shape:', (25000, 300))


Get the word index and then Create a key-value pair for word and word_id 

In [ ]:
import numpy as np
from keras.datasets.imdb import get_word_index
word_index = imdb.get_word_index()
embeddings_index = dict()
# create a weight matrix for words 
embedding_matrix = np.zeros((vocab_size, 100))
for word, index in word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[index] = embedding_vector


1654784/1641221 [==============================] - 1s 0us/step


## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [ ]:
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
# define the model
model = Sequential()
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen , trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 100)          1000000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 30000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 30001     
Total params: 1,030,001
Trainable params: 30,001
Non-trainable params: 1,000,000
_________________________________________________________________
None


Report the Accuracy of the model 

In [ ]:
# fit the model
history = model.fit(x_train, y_train, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 2s 120us/step - loss: 0.6932 - accuracy: 0.4976 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 2/10
20000/20000 [==============================] - 1s 38us/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 3/10
20000/20000 [==============================] - 1s 37us/step - loss: 0.6932 - accuracy: 0.4924 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 4/10
20000/20000 [==============================] - 1s 36us/step - loss: 0.6932 - accuracy: 0.4983 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 5/10
20000/20000 [==============================] - 1s 38us/step - loss: 0.6932 - accuracy: 0.5016 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 6/10
20000/20000 [==============================] - 1s 35us/step - loss: 0.6932 - accuracy: 0.4978 - val_loss: 0.6932 - val_accuracy: 0.4938
Epoch 7/10
20000/20000 [==============================] - 1s 37us/st

In [ ]:
score = model.evaluate(x_test, y_test, verbose=1)

25000/25000 [==============================] - 2s 64us/step


In [ ]:
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

('Test Score:', 0.6931502214622498)
('Test Accuracy:', 0.5)


In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 50.000000
